In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-03-30 03:10:52--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-03-30 03:10:53--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc721d67f1432c201498738bf400.dl.dropboxusercontent.com/cd/0/inline/BiaUh31HX8epcx0ljSkA5Hn7fYkjYV_yx1ngQJWkm1FwFuoFpXZi9x1Eh-JIGMQGh1aUMEWXXA1GzYJswtvufcv6iwJYWgbDEdLoICBP2pYPt6Uxw48H9QPtvzSSaT55eryzQePTiF5aPAtmSQeGTsSjZVi07w5f_TR6GjRwKfxNog/file# [following]
--2022-03-30 03:10:53--  https://uc721d67f1432c201498738bf400.dl.dropboxusercontent.com/cd/0/inline/BiaUh31HX8epcx0ljSkA5Hn7fY

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, RNN, SimpleRNNCell, Embedding, Dropout

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"This is a truly magnificent and heartwrenching film!!!! Ripstein's locations are spectacular, extremely detailed and well lit, the dialogue is extraordinarily Garc\xc3\xada M\xc3\xa1rquez, no doubt about it. Fernando Luj\xc3\xa1n and Marisa Paredes give us outstanding performances as the colonel and his wife.  You must see it!!!"
1


In [12]:
text_dataset_from_directory('movie-reviews-dataset/train')

Found 25000 files belonging to 2 classes.


<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
model = Sequential()

In [7]:
model.add(Input(shape=(1,), dtype="string"))

In [8]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [9]:
train_data

<MapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [10]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [13]:
train_data

<MapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
model.add(Embedding(max_tokens + 1, 128))

rnn = RNN(SimpleRNNCell(64) , return_sequences=False,return_state=False)
model.add(rnn)
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 63s 78ms/step - loss: 0.6938 - accuracy: 0.5165
Epoch 2/10
782/782 [==============================] - 60s 77ms/step - loss: 0.6854 - accuracy: 0.5494
Epoch 3/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6527 - accuracy: 0.6087
Epoch 4/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6632 - accuracy: 0.5975
Epoch 5/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6658 - accuracy: 0.5897
Epoch 6/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6394 - accuracy: 0.6265
Epoch 7/10
782/782 [==============================] - 59s 75ms/step - loss: 0.6597 - accuracy: 0.5989
Epoch 8/10
782/782 [==============================] - 60s 77ms/step - loss: 0.6384 - accuracy: 0.6301
Epoch 9/10
782/782 [==============================] - 61s 78ms/step - loss: 0.6140 - accuracy: 0.6589
Epoch 10/10
782/782 [==============================] - 60s 77ms/step - loss: 0.646

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 14s 18ms/step - loss: 0.7134 - accuracy: 0.5574


[0.7133914828300476, 0.5574399828910828]

In [ ]:
text = "I loved the movie !"

In [ ]:
model.predict([text])

array([[0.58057064]], dtype=float32)